In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from sqlalchemy import create_engine

DATA_PATH = '../data/raw/'

# Load all tables
tables = {}
for file in os.listdir(DATA_PATH):
    if file.endswith('.csv'):
        name = file.replace('olist_', '').replace('_dataset.csv', '').replace('.csv', '')
        tables[name] = pd.read_csv(DATA_PATH + file)

print("✓ All tables loaded")
print("Tables available:", list(tables.keys()))


✓ All tables loaded
Tables available: ['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation']


In [2]:
orders = tables['orders'].copy()

# Step 1: Keep only delivered orders
orders = orders[orders['order_status'] == 'delivered'].copy()
print(f"After filter: {len(orders):,} delivered orders")

# Step 2: Convert date columns to datetime
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])

# Step 3: Delivery time in days
orders['delivery_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_purchase_timestamp']
).dt.days

# Step 4: Late delivery flag
orders['is_late'] = (
    orders['order_delivered_customer_date'] >
    orders['order_estimated_delivery_date']
).astype(int)

# Step 5: Time features
orders['purchase_year']  = orders['order_purchase_timestamp'].dt.year
orders['purchase_month'] = orders['order_purchase_timestamp'].dt.month
orders['purchase_dow']   = orders['order_purchase_timestamp'].dt.day_name()

print(f"Late deliveries: {orders['is_late'].sum():,} ({orders['is_late'].mean()*100:.1f}%)")
print(f"Avg delivery days: {orders['delivery_days'].mean():.1f}")
print("✓ Orders cleaned")


After filter: 96,478 delivered orders
Late deliveries: 7,826 (8.1%)
Avg delivery days: 12.1
✓ Orders cleaned


In [3]:
## ── PAYMENTS ──────────────────────────────────────────
payments = tables['order_payments'].copy()

# Aggregate to one row per order
payments_agg = payments.groupby('order_id').agg(
    total_payment = ('payment_value', 'sum'),
    payment_type  = ('payment_type', 'first'),
    installments  = ('payment_installments', 'max')
).reset_index()

print(f"Payments aggregated: {len(payments_agg):,} orders")


## ── PRODUCTS ──────────────────────────────────────────
products = tables['products'].copy()
trans     = tables['product_category_name_translation'].copy()

# Translate category names
products = products.merge(trans, on='product_category_name', how='left')

# Fill missing category with 'unknown'
products['product_category_name_english'] = (
    products['product_category_name_english']
    .fillna('unknown')
)

print(f"Products cleaned: {len(products):,} products")


## ── REVIEWS ───────────────────────────────────────────
reviews = tables['order_reviews'].copy()

# Keep only order_id and review_score
reviews_clean = reviews[['order_id', 'review_score']].copy()

# Drop duplicate reviews
reviews_clean = reviews_clean.drop_duplicates(subset='order_id', keep='last')

print(f"Reviews cleaned: {len(reviews_clean):,} reviews")
print("✓ All tables cleaned")


Payments aggregated: 99,440 orders
Products cleaned: 32,951 products
Reviews cleaned: 98,673 reviews
✓ All tables cleaned


In [4]:
items     = tables['order_items'].copy()
customers = tables['customers'].copy()

# Aggregate items per order
items_agg = items.groupby('order_id').agg(
    item_count    = ('order_item_id', 'count'),
    total_price   = ('price', 'sum'),
    total_freight = ('freight_value', 'sum'),
    product_id    = ('product_id', 'first')
).reset_index()

# Build master table
master = (
    orders
    .merge(customers,    on='customer_id')
    .merge(payments_agg, on='order_id')
    .merge(items_agg,    on='order_id')
    .merge(reviews_clean, on='order_id', how='left')
    .merge(
        products[['product_id', 'product_category_name_english']],
        on='product_id',
        how='left'
    )
)

print(f"Master table shape: {master.shape}")
print(f"Number of columns: {len(master.columns)}")
master.head(2)


Master table shape: (96477, 26)
Number of columns: 26


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_days,is_late,...,customer_state,total_payment,payment_type,installments,item_count,total_price,total_freight,product_id,review_score,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,0,...,SP,38.71,credit_card,1,1,29.99,8.72,87285b34884572647811a353c7ac498a,4.0,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,0,...,BA,141.46,boleto,1,1,118.70,22.76,595fac2a385ac33a80bd5114aec74eb8,4.0,perfumery


In [5]:
# Inspect column names
for i, col in enumerate(master.columns, start=1):
    print(i, col)


1 order_id
2 customer_id
3 order_status
4 order_purchase_timestamp
5 order_approved_at
6 order_delivered_carrier_date
7 order_delivered_customer_date
8 order_estimated_delivery_date
9 delivery_days
10 is_late
11 purchase_year
12 purchase_month
13 purchase_dow
14 customer_unique_id
15 customer_zip_code_prefix
16 customer_city
17 customer_state
18 total_payment
19 payment_type
20 installments
21 item_count
22 total_price
23 total_freight
24 product_id
25 review_score
26 product_category_name_english


In [6]:
print("Delivered orders:", len(orders))
print("Master table rows:", len(master))
print("Difference:", len(orders) - len(master))


Delivered orders: 96478
Master table rows: 96477
Difference: 1


In [7]:
# Save master table
master.to_csv('../data/processed/master_orders.csv', index=False)

# Save individual cleaned tables
orders.to_csv('../data/processed/orders_clean.csv', index=False)
products.to_csv('../data/processed/products_clean.csv', index=False)
customers.to_csv('../data/processed/customers_clean.csv', index=False)

print("✓ Saved: data/processed/master_orders.csv")
print("✓ Saved: data/processed/orders_clean.csv")
print("✓ Saved: data/processed/products_clean.csv")
print("✓ Saved: data/processed/customers_clean.csv")
print(f"\nMaster table: {len(master):,} rows × {len(master.columns)} columns")


✓ Saved: data/processed/master_orders.csv
✓ Saved: data/processed/orders_clean.csv
✓ Saved: data/processed/products_clean.csv
✓ Saved: data/processed/customers_clean.csv

Master table: 96,477 rows × 26 columns


In [8]:
# Create SQLite database in data folder
engine = create_engine('sqlite:///../data/olist.db')

# Tables to load into SQL
tables_to_load = {
    'master_orders' : master,
    'orders'        : orders,
    'customers'     : customers,
    'products'      : products,
    'payments'      : payments_agg,
    'reviews'       : reviews_clean,
    'order_items'   : tables['order_items'],
    'sellers'       : tables['sellers'],
}

for table_name, df in tables_to_load.items():
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"✓ Loaded {table_name:<15} {len(df):>8,} rows")

print("\n✅ SQLite database ready at: data/olist.db")


✓ Loaded master_orders     96,477 rows
✓ Loaded orders            96,478 rows
✓ Loaded customers         99,441 rows
✓ Loaded products          32,951 rows
✓ Loaded payments          99,440 rows
✓ Loaded reviews           98,673 rows
✓ Loaded order_items      112,650 rows
✓ Loaded sellers            3,095 rows

✅ SQLite database ready at: data/olist.db


In [10]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('../data/olist.db')

# Monthly revenue query
query = """
SELECT
    strftime('%Y-%m', order_purchase_timestamp) AS month,
    COUNT(DISTINCT order_id)                    AS total_orders,
    ROUND(SUM(total_payment), 2)                AS revenue,
    ROUND(AVG(total_payment), 2)                AS avg_order_value
FROM master_orders
GROUP BY month
ORDER BY month
LIMIT 10;
"""

result = pd.read_sql_query(query, conn)

print(result.to_string(index=False))

conn.close()


  month  total_orders   revenue  avg_order_value
2016-10           265  46566.71           175.72
2016-12             1     19.62            19.62
2017-01           750 127545.67           170.06
2017-02          1653 271298.65           164.13
2017-03          2546 414369.39           162.75
2017-04          2303 390952.18           169.76
2017-05          3546 567066.73           159.92
2017-06          3135 490225.60           156.37
2017-07          3872 566403.93           146.28
2017-08          4193 646000.61           154.07
